In [1]:
import pyspark
import pyspark.sql
import numpy as np
import pyspark.ml

In [2]:
sc = pyspark.sql.SparkSession.builder.getOrCreate()

In [3]:
sc

In [4]:
df = sc.read.csv("data/PS_20174392719_1491204439457_log.csv", inferSchema=True, header=True).limit(int(1e6))

In [5]:
df.printSchema()

root
 |-- step: integer (nullable = true)
 |-- type: string (nullable = true)
 |-- amount: double (nullable = true)
 |-- nameOrig: string (nullable = true)
 |-- oldbalanceOrg: double (nullable = true)
 |-- newbalanceOrig: double (nullable = true)
 |-- nameDest: string (nullable = true)
 |-- oldbalanceDest: double (nullable = true)
 |-- newbalanceDest: double (nullable = true)
 |-- isFraud: integer (nullable = true)
 |-- isFlaggedFraud: integer (nullable = true)



In [6]:
df.show(5)

+----+--------+--------+-----------+-------------+--------------+-----------+--------------+--------------+-------+--------------+
|step|    type|  amount|   nameOrig|oldbalanceOrg|newbalanceOrig|   nameDest|oldbalanceDest|newbalanceDest|isFraud|isFlaggedFraud|
+----+--------+--------+-----------+-------------+--------------+-----------+--------------+--------------+-------+--------------+
|   1| PAYMENT| 9839.64|C1231006815|     170136.0|     160296.36|M1979787155|           0.0|           0.0|      0|             0|
|   1| PAYMENT| 1864.28|C1666544295|      21249.0|      19384.72|M2044282225|           0.0|           0.0|      0|             0|
|   1|TRANSFER|   181.0|C1305486145|        181.0|           0.0| C553264065|           0.0|           0.0|      1|             0|
|   1|CASH_OUT|   181.0| C840083671|        181.0|           0.0|  C38997010|       21182.0|           0.0|      1|             0|
|   1| PAYMENT|11668.14|C2048537720|      41554.0|      29885.86|M1230701703|      

In [7]:
df.count()

1000000

In [8]:
# Drop seemingly unnecessary columns
df = df.drop("nameOrig", "isFlaggedFraud", "nameDest")

In [9]:
df_train, df_test = df.randomSplit([0.9, 0.1], seed=16)

In [10]:
df_train.count()

899894

In [38]:
df_train.select("isFraud").groupBy("isFraud").count().show()

+-------+------+
|isFraud| count|
+-------+------+
|      0|899418|
|      1|   476|
+-------+------+



## Find out which columns are numerical and which ones are categorical

In [11]:
cat_cols = [col for col, dtype in df_train.dtypes if dtype == "string"]
num_cols = list(set(df_train.columns) - set(cat_cols) - {"isFraud", "isFlaggedFraud"})

print(cat_cols)
print(num_cols)

['type']
['oldbalanceOrg', 'amount', 'newbalanceDest', 'newbalanceOrig', 'oldbalanceDest', 'step']


## Data standardization

In [12]:
pipeline_num = pyspark.ml.Pipeline().setStages([
    pyspark.ml.feature.VectorAssembler(inputCols=num_cols, outputCol="vec_feats_num"),
    pyspark.ml.feature.StandardScaler(inputCol="vec_feats_num", outputCol="standardized")
])

print(type(pipeline_num))
pipeline_num = pipeline_num.fit(df_train)
print(type(pipeline_num))

df_train_std = pipeline_num.transform(df_train)
df_test_std = pipeline_num.transform(df_test)

<class 'pyspark.ml.pipeline.Pipeline'>
<class 'pyspark.ml.pipeline.PipelineModel'>


## Categorical features investigation and One-Hot Encoding

In [13]:
{c: df_train_std.select(c).distinct().count() for c in cat_cols}

{'type': 5}

In [14]:
df_train_std.groupBy("type").count().show()

+--------+------+
|    type| count|
+--------+------+
| CASH_IN|196881|
|CASH_OUT|326287|
|   DEBIT|  5792|
| PAYMENT|296681|
|TRANSFER| 74253|
+--------+------+



In [18]:
string_indexers = [
    pyspark.ml.feature.StringIndexer(inputCol=c, outputCol=f"{c}_si", handleInvalid="skip")
    for c in cat_cols
]

In [20]:
pipeline_cat = pyspark.ml.Pipeline().setStages([
    *string_indexers,
    pyspark.ml.feature.OneHotEncoder(
        inputCols=[f"{c}_si" for c in cat_cols],
        outputCols=[f"{c}_ohe" for c in cat_cols],
    )
])

print(type(pipeline_cat))
pipeline_cat = pipeline_cat.fit(df_train)
print(type(pipeline_cat))

df_train_std = pipeline_cat.transform(df_train_std)
df_test_std = pipeline_cat.transform(df_test_std)

<class 'pyspark.ml.pipeline.Pipeline'>
<class 'pyspark.ml.pipeline.PipelineModel'>


In [21]:
df_train_std.show(5)

+----+-------+------+-------------+--------------+--------------+--------------+-------+--------------------+--------------------+-------+-------------+
|step|   type|amount|oldbalanceOrg|newbalanceOrig|oldbalanceDest|newbalanceDest|isFraud|       vec_feats_num|        standardized|type_si|     type_ohe|
+----+-------+------+-------------+--------------+--------------+--------------+-------+--------------------+--------------------+-------+-------------+
|   1|CASH_IN|270.78|   4184966.65|    4185237.43|        3019.0|           0.0|      0|[4184966.65,270.7...|[1.40334656937633...|    2.0|(4,[2],[1.0])|
|   1|CASH_IN|484.57|   5422437.76|    5422922.33|    5638778.53|    5579568.65|      0|[5422437.76,484.5...|[1.81830826015129...|    2.0|(4,[2],[1.0])|
|   1|CASH_IN|783.31|   8150331.93|    8151115.24|       2013.12|       1229.81|      0|[8150331.93,783.3...|[2.73305412200689...|    2.0|(4,[2],[1.0])|
|   1|CASH_IN|863.08|   9290756.54|    9291619.62|       5577.88|        4714.8|  

In [22]:
df_test_std.show(5)

+----+-------+--------+-------------+--------------+--------------+--------------+-------+--------------------+--------------------+-------+-------------+
|step|   type|  amount|oldbalanceOrg|newbalanceOrig|oldbalanceDest|newbalanceDest|isFraud|       vec_feats_num|        standardized|type_si|     type_ohe|
+----+-------+--------+-------------+--------------+--------------+--------------+-------+--------------------+--------------------+-------+-------------+
|   1|CASH_IN|27070.11|    346803.59|      373873.7|       70595.0|     122750.49|      0|[346803.59,27070....|[0.11629378892993...|    2.0|(4,[2],[1.0])|
|   1|CASH_IN|30811.56|   7243810.89|    7274622.45|      152178.0|     651524.92|      0|[7243810.89,30811...|[2.42907005285034...|    2.0|(4,[2],[1.0])|
|   1|CASH_IN|31646.31|   5847973.23|    5879619.54|      58072.44|       66575.5|      0|[5847973.23,31646...|[1.96100324243328...|    2.0|(4,[2],[1.0])|
|   1|CASH_IN|32726.88|   1640596.99|    1673323.87|     102483.74|   

In [40]:
va = pyspark.ml.feature.VectorAssembler(inputCols=["standardized", "type_ohe"], outputCol="vec_feats_total")
df_train_aux = va.transform(df_train_std)

# This is a requirement of Spark (a vector column named 'features' and a double column named 'label').
data = df_train_aux.select(
    pyspark.sql.functions.col("vec_feats_total").alias("features"),
    pyspark.sql.functions.col("isFraud").alias("label"),
    
)

In [42]:
data.show(5, truncate=False)

+--------------------------------------------------------------------------------------------------------------------------------------------+-----+
|features                                                                                                                                    |label|
+--------------------------------------------------------------------------------------------------------------------------------------------+-----+
|[1.403346569376335,0.0010435331993796247,0.0,1.3862883161119874,0.0013116646729908526,0.07904243517400436,0.0,0.0,1.0,0.0]                  |0    |
|[1.818308260151296,0.001867438076753766,2.301952399993384,1.7962502703847307,2.4498796278967507,0.07904243517400436,0.0,0.0,1.0,0.0]        |0    |
|[2.733054122006899,0.003018723651695302,5.073804551245845E-4,2.699917509935478,8.746400750219758E-4,0.07904243517400436,0.0,0.0,1.0,0.0]    |0    |
|[3.1154731704539977,0.003326141641629982,0.0019451763848248034,3.0776900790937693,0.0024234210487519767,0

In [43]:
model = pyspark.ml.classification.LogisticRegression().fit(data)

In [45]:
model.summary.areaUnderROC

0.9852501267813298

In [46]:
model.summary.pr.show()

+------------------+--------------------+
|            recall|           precision|
+------------------+--------------------+
|               0.0|  0.5493333333333333|
|0.4327731092436975|  0.5493333333333333|
|0.5735294117647058| 0.37092391304347827|
|0.6491596638655462| 0.28167730173199634|
|0.6764705882352942| 0.22085048010973937|
|0.6953781512605042| 0.18196811434854315|
|0.6974789915966386| 0.15229357798165138|
|0.7121848739495799|  0.1334120425029516|
|0.7226890756302521| 0.11853893866299105|
|0.7247899159663865|  0.1057309224639902|
|0.7310924369747899| 0.09602649006622517|
|0.7373949579831933| 0.08808030112923464|
|0.7457983193277311| 0.08168430740911183|
|0.7478991596638656| 0.07563203739111961|
|0.7605042016806722| 0.07142857142857142|
|0.7710084033613446| 0.06759992632160619|
|0.7878151260504201| 0.06476683937823834|
|0.8046218487394958|0.062266298162900344|
|0.8130252100840336| 0.05942874692874693|
|0.8340336134453782|  0.0577622581114506|
+------------------+--------------